In [1]:
# Tensorflow imports
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

## Model parameters

In [2]:
model_name = 'faceDetectModelAugmentation'
savedModelPath = '../log/saved_models/faceDetectModelAugmentation'
tb_log_dir = "../log/tensorboard/faceDetectModelAugmentation"
cp_filepath = '../log/cps/faceDetectModelAugmentation/latest_weights.h5'

batch_size = 32
epochs = 10

doDataAugmentation = True
useDropout = True
dropoutRate = 0.25

## Model callbacks

In [3]:
callbacks = [
    # Checkpoint callback                    
    keras.callbacks.ModelCheckpoint(
                    filepath=cp_filepath, 
                    verbose=1, 
                    save_weights_only=True),

    # Tensorboard callback
    keras.callbacks.TensorBoard(log_dir=tb_log_dir, histogram_freq=1),

    # Early Stopping callback
    keras.callbacks.EarlyStopping(
                    monitor="val_loss",
                    patience=2,
                    verbose=1,
                )
]

## Data parameters

In [4]:
image_height = 224
image_width = 224

## Data Augmentation

In [5]:
data_augmentation = keras.Sequential(
    [
        keras.layers.RandomFlip("horizontal"),
        keras.layers.RandomRotation(0.1),
    ]
)

## Creating the training dataset

In [6]:
# https://www.tensorflow.org/api_docs/python/tf/keras/utils/image_dataset_from_directory

face_training_ds = keras.utils.image_dataset_from_directory(
    "../data/training/",
    label_mode='binary',
    color_mode='rgb',
    batch_size=batch_size,
    image_size=(image_height, image_width),
    shuffle=True,
    seed=123,
    validation_split=0.2,
    subset='training',
    interpolation='bilinear',
    follow_links=False,
    crop_to_aspect_ratio=True,
)

face_training_ds.cache().batch(batch_size).prefetch(buffer_size=10)

Found 28312 files belonging to 2 classes.
Using 22650 files for training.


<PrefetchDataset element_spec=(TensorSpec(shape=(None, None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, None, 1), dtype=tf.float32, name=None))>

## Creating the validation dataset

In [7]:
validation_ds = keras.utils.image_dataset_from_directory(
    "../data/training/",
    label_mode='binary',
    color_mode='rgb',
    batch_size=batch_size,
    image_size=(image_height, image_width),
    shuffle=True,
    seed=123,
    validation_split=0.2,
    subset='validation',
    interpolation='bilinear',
    follow_links=False,
    crop_to_aspect_ratio=True,
)

Found 28312 files belonging to 2 classes.
Using 5662 files for validation.


## Creating the test dataset

In [8]:
test_ds = keras.utils.image_dataset_from_directory(
    "../data/test/",
    label_mode='binary',
    color_mode='rgb',
    batch_size=batch_size,
    image_size=(image_height, image_width),
    interpolation='bilinear',
    follow_links=False,
    crop_to_aspect_ratio=True,
)

test_ds.cache().batch(batch_size).prefetch(buffer_size=10)

Found 1344 files belonging to 2 classes.


<PrefetchDataset element_spec=(TensorSpec(shape=(None, None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, None, 1), dtype=tf.float32, name=None))>

## Load the ResNet model

In [9]:
# Loading either the ResNet50 architecture model or the previously saved model, and freeze it for transfer learning
resnet = ResNet50(
                include_top=False,
                weights=None,  # No pretrained weights
                input_shape=(image_height, image_width, 3)
                )

## Adding Pooling and Fully Connected Layer for binary classification

In [10]:
inputs = keras.Input(shape=(image_height, image_width, 3))

# Data Augmentation on input
if(doDataAugmentation):
    inputs = data_augmentation(inputs)

# Running base model in inference mode
x = resnet(inputs, training=False)

# Flatten tensor
x = keras.layers.Flatten()(x)

# Add Dropout layer
if(useDropout):
    x = keras.layers.Dropout(dropoutRate)(x)

# Final layer for binary classification
outputs = keras.layers.Dense(1, activation='sigmoid')(x)

model = keras.Model(inputs, outputs)

model.summary()

# Loading latest saved weights into model
# if(os.path.isfile(cp_filepath)):
#     print('Loading weights...')
#     model.load_weights(filepath=cp_filepath)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 flatten (Flatten)           (None, 100352)            0         
                                                                 
 dense (Dense)               (None, 1)                 100353    
                                                                 
Total params: 23,688,065
Trainable params: 23,634,945
Non-trainable params: 53,120
_________________________________________________________________


## Training the model with the dataset

In [11]:
model.compile(optimizer=keras.optimizers.Adam(),
              loss=keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])
history = model.fit(
            face_training_ds, 
            epochs=epochs, 
            callbacks=callbacks, 
            validation_data=validation_ds
        )

Epoch 1/10
708/708 [==============================] - ETA: 0s - loss: 5.2473 - accuracy: 0.8807

KeyError: 'Failed to format this callback filepath: "../log/cps/{model_name}/latest_weights.h5". Reason: \'model_name\''

## Save the model

In [ ]:
model.save(savedModelPath)

INFO:tensorflow:Assets written to: ../log/saved_models/faceDetectModel\assets


INFO:tensorflow:Assets written to: ../log/saved_models/faceDetectModel\assets


## Test model with test dataset

In [ ]:
results = model.evaluate(test_ds)

print(f'Loss: {results[0]}; Accuracy: {results[1]}')

## Predict new images

In [ ]:
# https://www.tensorflow.org/tutorials/images/classification

img = tf.keras.utils.load_img(
    '../data/test/NoFace/antelope_10248.jpg', target_size=(image_height, image_width)
)
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

preds = model.predict(img_array)
score = preds[0][0]

if(score > 0.5):
    print("This image contains no face. ({:.2f} percent confidence)".format(100 * score))
else:
    print("This image contains a face. ({:.2f} percent confidence)".format(100 * (1-score)))

1/1 [==============================] - 0s 99ms/step
This image contains a face. (99.00 percent confidence)
